In [54]:
import pandas as pd
import numpy as np
from scipy import stats
from scipy.optimize import minimize 
from statsmodels.tsa.arima.model import ARIMA

Problem 1 - Price Return Calculation

Calculate and compare the expected value and standard deviation of price at time t (P𝑡) ,given each of the 3 types of price returns, assuming rt ~ N(0,σ^2). 

Simulate each return equation using rt ~ N(0,σ^2) and show the mean and standard deviation match your expectations.

Problem 1.1

Calculate and compare the expected value and standard deviation of price at time t (P𝑡) ,given each of the 3 types of price returns, assuming rt ~ N(0,σ^2). 

In [55]:
def exp_val_std(method, P0, sigma, mu):
    if method == 'Classical_Brownian':
        exp_val = P0 + mu
        exp_std = sigma
    elif method == 'Arithmetic':
        exp_val = P0 * (1 + mu)
        exp_std = sigma * P0
    elif method == 'Geometric_Brownian':
        exp_val = np.exp(mu + np.log(P0) + 0.5 * sigma ** 2)
        exp_std = np.sqrt(np.exp(sigma ** 2) - 1) * np.exp(mu + np.log(P0) + 0.5 * sigma ** 2)
    return exp_val, exp_std

In [86]:
P0 = 100
sigma = 0.1
mu = 0
methods = ["Classical_Brownian", "Arithmetic", "Geometric_Brownian"]
for method in methods:
    exp_val, exp_std = exp_val_std(method, P0, sigma, mu)
    print("For {} method, the expected value is {} and the expected standard deviation is {}.".format(method, exp_val, exp_std))

For Classical_Brownian method, the expected value is 100 and the expected standard deviation is 0.1.
For Arithmetic method, the expected value is 100 and the expected standard deviation is 10.0.
For Geometric_Brownian method, the expected value is 100.50125208594014 and the expected standard deviation is 10.0753029446204.


Problem 1.2

Simulate each return equation using rt ~ N(0,σ^2) and show the mean and standard deviation match your expectations.

In [87]:
def price_series(method, P0, t, sigma, mu):
    rt = np.random.normal(mu, sigma, t)
    P = np.zeros(t)
    if method == 'Classical_Brownian':
        for i in range(t):
            P[i] = P0 + rt[i]
    elif method == 'Arithmetic':
        for i in range(t):
            P[i] = P0 * (1 + rt[i])
    elif method == 'Geometric_Brownian':
        for i in range(t):
            P[i] = P0 * np.exp(rt[i])
    return P

In [88]:
t = 10000
np.random.seed(0)

for method in methods:
    P = price_series(method, P0, t, sigma, mu)
    exp_val, exp_std = exp_val_std(method, P0, sigma, mu)
    simu_val = np.mean(P)
    simu_std = np.std(P)
    print("{}".format(method))
    print("Theoretical Expacted Value is {}".format(exp_val))
    print("Simulated Expacted Value is {}".format(simu_val))
    print("Value diff is {}".format(exp_val - simu_val))
    print("Theoretical Standard Deviation is {}".format(exp_std))
    print("Simulated Standard Deviation is {}".format(simu_std))
    print("Standard Deviation diff is {}".format(exp_std - simu_std))
    print(" ")

Classical_Brownian
Theoretical Expacted Value is 100
Simulated Expacted Value is 99.99815662798417
Value diff is 0.0018433720158270717
Theoretical Standard Deviation is 0.1
Simulated Standard Deviation is 0.0987556568176121
Standard Deviation diff is 0.0012443431823879053
 
Arithmetic
Theoretical Expacted Value is 100
Simulated Expacted Value is 100.10988005872944
Value diff is -0.10988005872944484
Theoretical Standard Deviation is 10.0
Simulated Standard Deviation is 9.930546809674302
Standard Deviation diff is 0.06945319032569763
 
Geometric_Brownian
Theoretical Expacted Value is 100.50125208594014
Simulated Expacted Value is 100.44097996457594
Value diff is 0.0602721213641928
Theoretical Standard Deviation is 10.0753029446204
Simulated Standard Deviation is 10.023662034365971
Standard Deviation diff is 0.05164091025442907
 


Problem 2 - VaR Calculation

Implement a function similar to the “return_calculate()” in this week’s code. Allow the user to specify the method of return calculation.

Use DailyPrices.csv. Calculate the arithmetic returns for all prices.

Remove the mean from the series so that the mean(META)=0

Calculate VaR

Using a normal distribution.

Using a normal distribution with an Exponentially Weighted variance (λ = 0. 94)

Using a MLE fitted T distribution.

Using a fitted AR(1) model.

Using a Historic Simulation.

Compare the 5 values.

Problem 2.1

Implement a function similar to the “return_calculate()” in this week’s code. Allow the user to specify the method of return calculation.

In [59]:
def return_calculate(prices, method="DISCRETE", dateColumn="Date"):
    vars = prices.columns
    nVars = len(vars)
    vars = [var for var in vars if var != dateColumn]
    if nVars == len(vars):
        raise ValueError("dateColumn: " + dateColumn + " not in DataFrame: " + str(vars))
    nVars = nVars - 1

    p = np.matrix(prices[vars])
    n = p.shape[0]
    m = p.shape[1]
    p2 = np.empty((n-1,m))

    for i in range(n-1):
        for j in range(m):
            p2[i,j] = p[i+1,j] / p[i,j]

    if method.upper() == "DISCRETE":
        p2 = p2 - 1.0
    elif method.upper() == "LOG":
        p2 = np.log(p2)
    else:
        raise ValueError("method: " + method + " must be in (\"LOG\",\"DISCRETE\")")

    dates = prices[dateColumn][1:n]
    out = pd.DataFrame({dateColumn: dates})
    for i in range(nVars):
        out[vars[i]] = p2[:,i]
    return out

Problem 2.2

Use DailyPrices.csv. Calculate the arithmetic returns for all prices.

In [60]:
prices = pd.read_csv('DailyPrices.csv')
returns = return_calculate(prices)

/var/folders/mx/h47vz6_d3cj1b2c96lcwngfh0000gn/T/ipykernel_1255/2906147436.py:28: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  out[vars[i]] = p2[:,i]


Problem 2.3

Remove the mean from the series so that the mean(META)=0

In [61]:
meta = returns['META'] - returns['META'].mean()

Problem 2.4

Calculate VaR

Using a normal distribution.

Using a normal distribution with an Exponentially Weighted variance (λ = 0. 94)

Using a MLE fitted T distribution.

Using a fitted AR(1) model.

Using a Historic Simulation.

Compare the 5 values.

Problem 2.4.1

Using a normal distribution.

In [62]:
def cal_VaR_norm(returns, n=10000, alpha=0.05):
    mu = returns.mean()
    sigma = returns.std()
    simu_returns = np.random.normal(mu, sigma, n)
    simu_returns.sort()
    VaR_normal = -np.percentile(simu_returns, alpha*100)
    return VaR_normal

In [63]:
VaR_normal = cal_VaR_norm(meta)
print("VaR_normal is " + str(VaR_normal))

VaR_normal is 0.06683120453479682


Problem 2.4.2

Using a normal distribution with an Exponentially Weighted variance (λ = 0. 94)

In [64]:
def exp_weighted_cov(input, lambda_=0.97):
    ror = input.values
    ror_mean = np.mean(ror, axis=0)
    dev = ror - ror_mean
    times = dev.shape[0]
    weights = np.zeros(times)
    
    for i in range(times):
        weights[times - i - 1]  = (1 - lambda_) * lambda_**i
    
    weights_mat = np.diag(weights/sum(weights))

    cov = np.transpose(dev) @ weights_mat @ dev
    return cov

In [65]:
def cal_VaR_ew_norm(returns, lambda_=0.94, n=10000, alpha=0.05):
    mu = returns.mean()
    sigma = np.sqrt(exp_weighted_cov(returns, lambda_=lambda_))
    simu_returns = np.random.normal(mu, sigma, n)
    VaR_ew = -np.percentile(simu_returns, alpha*100)
    return VaR_ew

In [66]:
VaR_ew = cal_VaR_ew_norm(meta)
print("VaR_ew is " + str(VaR_ew))

VaR_ew is 0.09221037744633959


Problem 2.4.3

Using a MLE fitted T distribution.

In [67]:
def MLE_t(params, returns):
    df, loc, scale = params
    neg_LL = -1 * np.sum(stats.t.logpdf(returns, df=df, loc=loc, scale=scale))
    return(neg_LL)

In [68]:
def cal_VaR_MLE_t(returns, n=10000, alpha=0.05):
    constraints = [
        {'type': 'ineq', 'fun': lambda x: x[0] - 1},
        {'type': 'ineq', 'fun': lambda x: x[2]}
    ]
    
    res = minimize(MLE_t, x0=[10, returns.mean(), returns.std()], args=(returns,), constraints=constraints)
    
    df, loc, scale = res.x
    sim_returns = stats.t.rvs(df, loc=loc, scale=scale, size=n)
    var_t = -np.percentile(sim_returns, alpha * 100)
    
    return var_t

In [69]:
VaR_MLE_t = cal_VaR_MLE_t(meta)
print("VaR_MLE_t is " + str(VaR_MLE_t))

VaR_MLE_t is 0.056471534942947196


Problem 2.4.4

Using a fitted AR(1) model.

In [70]:
def cal_VaR_AR1(returns, n=10000, alpha=0.05):
    #a more general model that extends the ARMA model to non-stationary time series data.
    model = ARIMA(returns, order=(1, 0, 0)).fit()
    sigma = np.std(model.resid)
    sim_returns = np.empty(n)
    returns = returns.values
    for i in range(n):
        sim_returns[i] =  model.params[0] * (returns[-1]) + sigma * np.random.normal()
    VaR_AR1 = -np.percentile(sim_returns, alpha*100)
    return VaR_AR1

In [71]:
VaR_AR1 = cal_VaR_AR1(meta)
print("VaR_AR1 is " + str(VaR_AR1))

VaR_AR1 is 0.06532885399515062


Problem 2.4.5

Using a Historic Simulation.

In [72]:
def cal_VaR_hist(returns, alpha=0.05):
    VaR_hist = -np.percentile(returns, alpha*100)
    return VaR_hist

In [73]:
VaR_hist = cal_VaR_hist(meta)
print("VaR_hist is " + str(VaR_hist))

VaR_hist is 0.0546200790823787


Problem 3 - Portfolio VaR Calculation
Using Portfolio.csv and DailyPrices.csv. Assume the expected return on all stocks is 0.

This file contains the stock holdings of 3 portfolios. You own each of these portfolios. Using an exponentially weighted covariance with lambda = 0.94, calculate the VaR of each portfolio as well as your total VaR (VaR of the total holdings). Express VaR as a $.

Discuss your methods and your results.

Choose a different model for returns and calculate VaR again. Why did you choose that model? How did the model change affect the results?

Problem 3.1 - Portfolio VaR Calculation
Using Portfolio.csv and DailyPrices.csv. Assume the expected return on all stocks is 0.

This file contains the stock holdings of 3 portfolios. You own each of these portfolios. Using an exponentially weighted covariance with lambda = 0.94, calculate the VaR of each portfolio as well as your total VaR (VaR of the total holdings). Express VaR as a $.

In [74]:
portfolio = pd.read_csv("portfolio.csv")

In [75]:
def get_single_portfolio(portfolio, prices, portfolio_code):
    assets = portfolio[portfolio["Portfolio"] == portfolio_code]
        
    codes = list(assets["Stock"])
    assets_prices = prices[["Date"] + codes].copy()
    
    Total_Value = prices[codes].tail(1).values.dot(assets["Holding"].values)[0]
    holdings = assets["Holding"].values

    asset_values = holdings.reshape(-1, 1) * prices[codes].tail(1).T.values
    delta = asset_values / Total_Value
    
    return Total_Value, assets_prices, holdings, delta


In [76]:
def get_all_portfolio(portfolio, prices):
    assets = portfolio.drop('Portfolio',axis=1)
    assets = assets.groupby(["Stock"], as_index=False)["Holding"].sum()
        
    codes = list(assets["Stock"])
    assets_prices = prices[["Date"] + codes].copy()
    
    Total_Value = prices[codes].tail(1).values.dot(assets["Holding"].values)[0]
    holdings = assets["Holding"].values

    asset_values = holdings.reshape(-1, 1) * prices[codes].tail(1).T.values
    delta = asset_values / Total_Value
    
    return Total_Value, assets_prices, holdings, delta

In [77]:
Total_Value_A, assets_prices_A, holdings_A, delta_A = get_single_portfolio(portfolio, prices, "A")
Total_Value_B, assets_prices_B, holdings_B, delta_B = get_single_portfolio(portfolio, prices, "B")
Total_Value_C, assets_prices_C, holdings_C, delta_C = get_single_portfolio(portfolio, prices, "C")
Total_Value_All, assets_prices_All, holdings_All, delta_All = get_all_portfolio(portfolio, prices)

In [78]:
def cal_delta_VaR(Total_Value, assets_prices, delta, alpha=0.05, lambda_=0.94):
    returns = return_calculate(assets_prices).drop('Date', axis=1)
    assets_cov = exp_weighted_cov(returns, lambda_)
    
    delta_norm_VaR = -Total_Value * stats.norm.ppf(alpha) * np.sqrt(delta.T @ assets_cov @ delta)
    
    return delta_norm_VaR.item()

In [79]:
delta_var_A = cal_delta_VaR(Total_Value_A, assets_prices_A, delta_A)
delta_var_B = cal_delta_VaR(Total_Value_B, assets_prices_B, delta_B)
delta_var_C = cal_delta_VaR(Total_Value_C, assets_prices_C, delta_C)
delta_var_All = cal_delta_VaR(Total_Value_All, assets_prices_All, delta_All)

In [80]:
print("Delta Normal VaR of portfolio A is ${}".format(delta_var_A))
print("Delta Normal VaR of portfolio B is ${}".format(delta_var_B))
print("Delta Normal VaR of portfolio C is ${}".format(delta_var_C))
print("Delta Normal VaR of All is ${}".format(delta_var_All))

Delta Normal VaR of portfolio A is $5670.202920147334
Delta Normal VaR of portfolio B is $4494.598410778259
Delta Normal VaR of portfolio C is $3786.5890108090503
Delta Normal VaR of portfolio All is $13577.07541897708


Problem 3.2

Choose a different model for returns and calculate VaR again. Why did you choose that model? How did the model change affect the results?

In [83]:
def cal_hist_VaR(Total_Value, assets_prices, holdings, alpha=0.05):
    returns = return_calculate(assets_prices).drop("Date", axis=1)
    assets_prices = assets_prices.drop('Date',axis=1)
    simu_returns = returns.sample(1000, replace=True)
    simu_prices = np.dot(simu_returns* assets_prices.tail(1).values.reshape(assets_prices.shape[1],),holdings)

    hist_VaR = -np.percentile(simu_prices, alpha*100)

    return hist_VaR

In [84]:
hist_var_A = cal_hist_VaR(Total_Value_A, assets_prices_A, holdings_A)
hist_var_B = cal_hist_VaR(Total_Value_B, assets_prices_B, holdings_B)
hist_var_C = cal_hist_VaR(Total_Value_C, assets_prices_C, holdings_C)
hist_var_All = cal_hist_VaR(Total_Value_All, assets_prices_All, holdings_All)

In [85]:
print("Historical VaR of portfolio A is ${}".format(hist_var_A))
print("Historical VaR of portfolio B is ${}".format(hist_var_B))
print("Historical VaR of portfolio C is ${}".format(hist_var_C))
print("Historical VaR of All is ${}".format(hist_var_All))

Historical VaR of portfolio A is $9005.067216160829
Historical VaR of portfolio B is $7273.769164086196
Historical VaR of portfolio C is $5773.472215034216
Historical VaR of portfolio All is $21103.398010768073
